In [0]:
import numpy as np
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling3D, Flatten, Conv3D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import random
from os import walk


Actions = np.array([np.array([0, 1, 0]),
np.array([0, 0, 0]),
np.array([0, 0, 0.4]),
np.array([-1, 0, 0]),
np.array([1, 0, 0]),
np.array([-1, 0, 0.4]),
np.array([1, 0, 0.4])])

In [0]:
walking = walk('/content/drive/My Drive/Gym') #собираем имена файлов, содержащих инфу о прохождении
for i in walking:
  names = i[2]
  break

In [7]:
import pandas as pd

states_expanded = []    # это всё переводит собранные данные в формат удобный для обучения сети
actions_expanded = []
actions_one_hot_expanded = []
v = 0
for j in names:
  data = pd.read_pickle('/content/drive/My Drive/Gym/{}'.format(j))
  states = data['states']
  actions = data['actions']
  actions_one_hot = actions_one_hot = data['actions_onehot']
  for i in range(55, len(states)):
    states_expanded.append(np.array(states[i]))
    actions_expanded.append(np.array(actions[i]))
    actions_one_hot_expanded.append(np.array(actions_one_hot[i]))
actions_expanded = np.array(actions_expanded)
actions_one_hot_expanded = np.array(actions_one_hot_expanded)
states_expanded = np.array(states_expanded)
states_expanded.shape

(95541, 96, 96, 3)

In [0]:
# X_train = np.array(states_expanded[:50000])
# del states_expanded[:50000]
# X_test = np.array(states_expanded[:30000])
# del states_expanded[:30000]
# X_train = np.append(X_train, X_test, axis=0)
# X_test = None

In [0]:
# X_test = np.array(states_expanded)
# states_expanded = None

In [0]:
data = None
states = None
actions = None
actions_one_hot = None
# states_expanded = np.array(states_expanded)

In [0]:
def create_model():   #создаём модель
  inputs = Input(shape=(96,96,3))
  X = Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu')(inputs)
  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.5)(X)
  X = Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='relu')(X)
  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.5)(X)
  X = Conv2D(filters=128, kernel_size=(5,5), padding='same', activation='relu')(X)
  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.3)(X)
  X = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(X)
  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.3)(X)
  X = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(X)
  X = MaxPooling2D((2,2))(X)
  X = Flatten()(X)
  X = Dense(64, activation='relu')(X)
  X = Dense(16, activation='relu')(X)
  outputs = Dense(7, activation='softmax')(X)
  model = Model(inputs=inputs,outputs=outputs)
  return model

In [65]:
model = create_model() #инициализируем модель
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 96, 96, 32)        2432      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 48, 48, 64)        51264     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 24, 24, 64)        0  

In [0]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['categorical_crossentropy']) #компилируем модель

In [67]:
history = model.fit(states_expanded, actions_one_hot_expanded,  batch_size=64, epochs=4, shuffle=False)  #обучаем модель

Epoch 1/4
1493/1493 [==============================] - 96s 64ms/step - loss: 1.6152 - categorical_crossentropy: 1.6152
Epoch 2/4
1493/1493 [==============================] - 96s 64ms/step - loss: 0.8354 - categorical_crossentropy: 0.8354
Epoch 3/4
1493/1493 [==============================] - 96s 64ms/step - loss: 0.7829 - categorical_crossentropy: 0.7829
Epoch 4/4
1493/1493 [==============================] - 96s 64ms/step - loss: 0.7684 - categorical_crossentropy: 0.7684


In [0]:
model.save('/content/drive/My Drive/Gym2/my_model.h5') #сохраняем модель